In [3]:
# Libraries for data loading and manipulation
import os, cv2, json, subprocess, zipfile, pickle, csv, re
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import pyarrow as pa

import torch
import torch.nn as nn

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

import pyarrow.parquet as pq

In [4]:
# Load static paths
root = '/media/sam/New Volume/Xenium_Data'
output_root = '/media/sam/Data2/baysor_rbpms_consolidated'
os.makedirs(output_root, exist_ok=True) # Make sure output path exists
xen_roots = ['output-XETG00230__0018429__Region_1__20240105__233208',
             'output-XETG00230__0018432__Region_2__20240105__233208',
             'BudoffRun2_Slide 3_4/BudoffRun2_Slide 3_4/output-XETG00230__0018336__Region_1__20240124__002923',
             'BudoffRun2_Slide 3_4/BudoffRun2_Slide 3_4/output-XETG00230__0018521__Region_1__20240124__002923',
             'BudoffRun3_Slide 5_6/BudoffRun3_Slide 5_6/output-XETG00230__0018624__Region_1__20240127__000149',
             'BudoffRun3_Slide 5_6/BudoffRun3_Slide 5_6/output-XETG00230__0022826__Region_1__20240127__000149',
             'BudoffRun4_Slide 7_8/BudoffRun4_Slide 7_8/output-XETG00230__0018300__Region_1__20240206__235339',
             'BudoffRun4_Slide 7_8/BudoffRun4_Slide 7_8/output-XETG00230__0022825__Region_1__20240206__235339']
coords_root = '/media/sam/Data2/xenium_rbpms_coordinates'

# Extract slide_ids
slides = [xen_root[-33:-28] for xen_root in xen_roots]

# Iterate through each slide directory
saved_paths = {}
for slide_id in slides:
    slide_path = os.path.join(output_root, slide_id)
    
    # Skip if slide directory doesn't exist
    if not os.path.exists(slide_path):
        continue
    
    # Get all subdirectories for this slide
    pqs = [os.path.join(slide_path, d,'transcripts.parquet')  for d in os.listdir(slide_path) if os.path.isdir(os.path.join(slide_path, d))]
    
    for pq in pqs:
        slice_id = pq[-33:-28]
        saved_paths[(slide_id, slice_id)] = pq

print(saved_paths)

{('18429', 'R1.01'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.01_region0/transcripts.parquet', ('18429', 'R1.02'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.02_region0/transcripts.parquet', ('18429', 'R1.03'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.03_region0/transcripts.parquet', ('18429', 'R1.04'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.04_region0/transcripts.parquet', ('18429', 'R1.05'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.05_region0/transcripts.parquet', ('18429', 'R1.06'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.06_region0/transcripts.parquet', ('18429', 'R1.07'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.07_region0/transcripts.parquet', ('18429', 'R1.08'): '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.08_region0/transcripts.parquet', ('18432', 'R1.09'): '/media/sam/Data2/baysor_rbpms_consolidated/18432/18432_R1.09_region0/transcripts.p

In [ ]:
import pyarrow.parquet as pq

parq_path = '/media/sam/Data2/baysor_rbpms_consolidated/18429/18429_R1.08_region0/transcripts.parquet'
test = pq.read_table(parq_path).to_pandas()
print(test.shape)

In [ ]:
test2 = pd.unique(test)
test2.shape

In [5]:
for ret_id, region_id in saved_paths:
    # Rebuild paths for specific iteration
    parq_path = saved_paths[(ret_id, region_id)]
    output_path = saved_paths[(ret_id, region_id)][:-19]
    baysor_seg_path = os.path.join(output_path, 'segmentation_polygons_2d.json')
    try:
        if not os.path.isfile(baysor_seg_path):
            # Construct the Baysor command
            baysor_command = f'baysor run "{parq_path}" -c "/home/sam/baysor_analysis/xenium.toml" -o "{output_path}" --prior-segmentation-confidence=.2 :cell_id'

            # Execute the Baysor command using subprocess
            print(f"Running Baysor for ROI {output_path}...")
            subprocess.run("export JULIA_NUM_THREADS=20", shell=True, check=True)
            subprocess.run(baysor_command, shell=True, check=True)
        else:
            print(f'{ret_id} region {region_id} already completed')
    except:
        print(f'{ret_id} region {region_id} skipped')

18429 region R1.01 already completed
18429 region R1.02 already completed
18429 region R1.03 already completed
18429 region R1.04 already completed
18429 region R1.05 already completed
18429 region R1.06 already completed
18429 region R1.07 already completed
18429 region R1.08 already completed
18432 region R1.09 already completed
18432 region R1.10 already completed
18432 region R1.11 already completed
18432 region R1.12 already completed
18336 region R2.01 already completed
18336 region R2.02 already completed
18336 region R2.03 already completed
18336 region R2.04 already completed
18336 region R2.05 already completed
18336 region R2.06 already completed
18336 region R2.07 already completed
Running Baysor for ROI /media/sam/Data2/baysor_rbpms_consolidated/18521/18521_R2.09_region0/...
[01:11:32] Info: Run R5f803545f
[01:11:32] Info: (2024-11-11) Run Baysor v0.7.0
[01:11:32] Info: Loading data...
[01:11:50] Warning: Genes NegControl*,BLANK_*,antisense_* are missing from the data
└ Ba

Progress:   0%|                                         |  ETA: 4 days, 22:26:53
  Iteration:                  2
  Max. difference:            0.823
raction of probs changed:  0.995


Progress:   0%|                                         |  ETA: 5 days, 0:49:37
  Iteration:                  3
  Max. difference:            0.775
raction of probs changed:  0.995


Progress:   0%|                                         |  ETA: 5 days, 0:36:41
  Iteration:                  4
  Max. difference:            0.778
raction of probs changed:  0.995


Progress:   0%|                                         |  ETA: 5 days, 0:41:46
  Iteration:                  5
  Max. difference:            0.751
raction of probs changed:  0.995


Progress:   0%|                                         |  ETA: 5 days, 0:48:43
  Iteration:                  6
  Max. difference:            0.748
raction of probs changed:  0.995


Progress:   0%|                                         |  ETA: 5 days, 0:54:13
  It

raction of probs changed:  0.866


Progress:   0%|                                         |  ETA: 5 days, 1:54:16
  Iteration:                  44
  Max. difference:            0.474
raction of probs changed:  0.859


Progress:   0%|                                         |  ETA: 5 days, 1:54:27
  Iteration:                  45
  Max. difference:            0.317
raction of probs changed:  0.851


Progress:   0%|                                         |  ETA: 5 days, 1:54:02
  Iteration:                  46
  Max. difference:            0.366
raction of probs changed:  0.844


Progress:   0%|                                         |  ETA: 5 days, 1:53:40
  Iteration:                  47
  Max. difference:            0.458
raction of probs changed:  0.841


Progress:   0%|                                         |  ETA: 5 days, 1:53:42
  Iteration:                  48
  Max. difference:            0.622
raction of probs changed:  0.837


Progress:   0%|                              

raction of probs changed:  0.618


Progress:   0%|                                         |  ETA: 5 days, 1:01:14
  Iteration:                  86
  Max. difference:            0.349
raction of probs changed:  0.616


Progress:   0%|                                         |  ETA: 5 days, 1:05:41
  Iteration:                  87
  Max. difference:            0.296
raction of probs changed:  0.614


Progress:   0%|                                         |  ETA: 5 days, 1:10:26
  Iteration:                  88
  Max. difference:            0.427
raction of probs changed:  0.611


Progress:   0%|                                         |  ETA: 5 days, 1:14:50
  Iteration:                  89
  Max. difference:            0.256
raction of probs changed:  0.605


Progress:   0%|                                         |  ETA: 5 days, 1:19:15
  Iteration:                  90
  Max. difference:            0.28
raction of probs changed:  0.596


Progress:   0%|                               

raction of probs changed:  0.462


Progress:   0%|                                         |  ETA: 5 days, 3:31:25
  Iteration:                  127
  Max. difference:            0.187
raction of probs changed:  0.458


Progress:   0%|                                         |  ETA: 5 days, 3:33:33
  Iteration:                  128
  Max. difference:            0.249
raction of probs changed:  0.451


Progress:   0%|                                         |  ETA: 5 days, 3:35:42
  Iteration:                  129
  Max. difference:            0.281
raction of probs changed:  0.444


Progress:   0%|                                         |  ETA: 5 days, 3:37:39
  Iteration:                  130
  Max. difference:            0.28
raction of probs changed:  0.436


Progress:   0%|                                         |  ETA: 5 days, 3:39:37
  Iteration:                  131
  Max. difference:            0.281
raction of probs changed:  0.429


Progress:   0%|                          

...

[23:11:52] Info: Processing complete.
[23:13:43] Info: Estimating local colors
[23:31:45] Info: Estimating boundary polygons
[23:34:34] Warning: To many values of z (6464705). Binning z-stack into 10 layers for polygon estimation.
└ Baysor.Processing /home/viktor_petukhov/.julia/dev/Baysor/src/processing/data_processing/boundary_estimation.jl:330


Progress: 100%|█████████████████████████████████████████| Time: 0:04:10


[23:38:52] Info: Saving results to /media/sam/Data2/baysor_rbpms_consolidated/22826/22826_R4.09_region0/segmentation.csv


Progress: 100%|█████████████████████████████████████████| Time: 0:00:07


[23:40:44] Info: All done!
22826 region R4.10 already completed
22826 region R4.11 already completed
22826 region R4.12 already completed
18300 region R4.13 already completed
18300 region R4.14 already completed
18300 region R5.01 already completed
18300 region R5.02 already completed
18300 region R5.03 already completed
18300 region R5.04 already completed
18300 region R5.06 already completed
18300 region R5.07 already completed
18300 region R5.16 already completed
18300 region R5.17 already completed
22825 region R3.09 already completed
22825 region R3.10 already completed
22825 region R3.13 already completed
22825 region R3.14 already completed
22825 region R4.01 already completed
22825 region R4.02 already completed
22825 region R4.03 already completed
22825 region R4.04 already completed
22825 region R5.15 already completed
